In [5]:
pip install numpy


   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.6 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.6 MB 4.9 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/12.6 MB 5.8 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.6 MB 6.3 MB/s eta 0:00:02
   ------------------- -------------------- 6.3/12.6 MB 6.5 MB/s eta 0:00:01
   ------------------------ --------------- 7.9/12.6 MB 6.7 MB/s eta 0:00:01
   ----------------------------- ---------- 9.4/12.6 MB 6.8 MB/s eta 0:00:01
   ---------------------------------- ----- 11.0/12.6 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.6 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install flask tensorflow h5py


  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl (7.5 kB)
  Using cached h5py-3.12.1-cp311-cp311-win_amd64.whl (3.0 MB)
                                              0.0/390.2 MB ? eta -:--:--
                                              0.1/390.2 MB 3.5 MB/s eta 0:01:51
                                              0.6/390.2 MB 7.6 MB/s eta 0:00:52
                                             1.1/390.2 MB 10.1 MB/s eta 0:00:39
                                              1.4/390.2 MB 9.0 MB/s eta 0:00:44
                                             2.3/390.2 MB 11.3 MB/s eta 0:00:35
                                             2.9/390.2 MB 11.5 MB/s eta 0:00:34
                                             3.5/390.2 MB 11.9 MB/s eta 0:00:33
                                             3.7/390.2 MB 10.8 MB/s eta 0:00:36
                                             4.5/390.2 MB 11.6 MB/s eta 0:00:34
                                             5.4/390.2 MB 12.3 MB/s eta 0:00:


[notice] A new release of pip is available: 23.1.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import os
import numpy as np
from flask import Flask, request, render_template
from tensorflow.keras.models import load_model
from PIL import Image
import base64
from io import BytesIO

app = Flask(__name__)

# Load the trained model
model_path = "KT_model.h5"
model = load_model(model_path, compile=False)

# Define class labels
CLASS_LABELS = ["Tumor", "Normal", "Stone", "Cyst"]

# Define allowed image extensions
ALLOWED_EXTENSIONS = {"png", "jpg", "jpeg"}

def allowed_file(filename):
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTENSIONS

def preprocess_image(image_file):
    """Preprocess the uploaded image for prediction."""
    try:
        image = Image.open(image_file).convert("RGB")  # Ensure 3 channels (RGB)
        image = image.resize((62, 62))  # Resize to match model input
        image_array = np.array(image) / 255.0  # Normalize pixel values
        image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

        print(f"✅ Preprocessed Image Shape: {image_array.shape}")  # Debugging
        return image_array
    except Exception as e:
        print(f"❌ Error processing image: {e}")
        return None

@app.route("/", methods=["GET", "POST"])
def home():
    prediction_text = None
    uploaded_image = None  # To store base64 image

    if request.method == "POST":
        if "file" not in request.files:
            return render_template("index2.html", error="No file uploaded.")

        file = request.files["file"]
        if file.filename == "":
            return render_template("index2.html", error="No selected file.")

        if file and allowed_file(file.filename):
            image_array = preprocess_image(file)
            if image_array is None:
                return render_template("index2.html", error="Image processing failed.")

            try:
                prediction = model.predict(image_array)
                probabilities = {CLASS_LABELS[i]: float(prediction[0][i]) for i in range(len(CLASS_LABELS))}

                # Sort predictions by confidence
                sorted_probs = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)
                predicted_label = sorted_probs[0][0]  # Highest confidence class
                confidence = sorted_probs[0][1] * 100  # Convert to percentage

                prediction_text = f"Top Prediction: {predicted_label} (Confidence: {confidence:.2f}%)"
                
                # Debugging: Print all class probabilities
                print("🔍 Model Prediction Probabilities:", sorted_probs)

                # Convert image to base64 for display
                buffered = BytesIO()
                image = Image.open(file)
                image.save(buffered, format="JPEG")
                uploaded_image = base64.b64encode(buffered.getvalue()).decode("utf-8")

            except Exception as e:
                print(f"❌ Error during prediction: {e}")
                return render_template("index2.html", error="Prediction failed.")

    return render_template("index2.html", prediction=prediction_text, uploaded_image=uploaded_image)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.10.6:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Feb/2025 12:31:27] "GET / HTTP/1.1" 200 -


✅ Preprocessed Image Shape: (1, 62, 62, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
🔍 Model Prediction Probabilities: [('Stone', 0.9944043159484863), ('Normal', 0.004350675269961357), ('Cyst', 0.0009067041100934148), ('Tumor', 0.0003383775765541941)]


INFO:werkzeug:127.0.0.1 - - [07/Feb/2025 12:31:38] "POST / HTTP/1.1" 200 -
